In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.models import Model
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  data_format="channels_first",
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [21]:
# manual implementation of resnet20v1

input_shape = (1, 100, 100)
inputs = Input(shape=input_shape)                                     #output (1, 100, 100)
y = resnet_layer(inputs=inputs, num_filters=16, strides=1)            #output (16, 100, 100)

#stack 0
x = resnet_layer(inputs=y, num_filters=16, strides=1)
x = resnet_layer(inputs=x, num_filters=16, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=16, strides=1)
x = resnet_layer(inputs=x, num_filters=16, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=16, strides=1)
x = resnet_layer(inputs=x, num_filters=16, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

#stack 1
x = resnet_layer(inputs=y, num_filters=32, strides=2)
x = resnet_layer(inputs=x, num_filters=32, strides=1, activation=None)
#linear projection
y = resnet_layer(inputs=y, num_filters=32, kernel_size=1, strides=2, activation=None, batch_normalization=False)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=32, strides=1)
x = resnet_layer(inputs=x, num_filters=32, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=32, strides=1)
x = resnet_layer(inputs=x, num_filters=32, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

#stack 2
x = resnet_layer(inputs=y, num_filters=64, strides=2)
x = resnet_layer(inputs=x, num_filters=64, strides=1, activation=None)
#linear projection
y = resnet_layer(inputs=y, num_filters=64, kernel_size=1, strides=2, activation=None, batch_normalization=False)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=64, strides=1)
x = resnet_layer(inputs=x, num_filters=64, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=64, strides=1)
x = resnet_layer(inputs=x, num_filters=64, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = AveragePooling2D(pool_size=8, data_format='channels_first')(y)
y = Flatten()(x)
outputs = Dense(1, activation='sigmoid', kernel_initializer='he_normal')(y)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 1, 100, 100)  0                                            
__________________________________________________________________________________________________
conv2d_222 (Conv2D)             (None, 16, 100, 100) 160         input_13[0][0]                   
__________________________________________________________________________________________________
batch_normalization_199 (BatchN (None, 16, 100, 100) 400         conv2d_222[0][0]                 
__________________________________________________________________________________________________
activation_199 (Activation)     (None, 16, 100, 100) 0           batch_normalization_199[0][0]    
__________________________________________________________________________________________________
conv2d_223

In [25]:
#resnet20v2

input_shape = (1, 100, 100)
inputs = Input(shape=input_shape)

x = resnet_layer(inputs=inputs, num_filters=16, conv_first=True)

# stage 0
# bottleneck residual unit
y = resnet_layer(inputs=x, num_filters=16, kernel_size=1, strides=1, activation=None, batch_normalization=False, conv_first=False)
y = resnet_layer(inputs=y, num_filters=16, conv_first=False)
y = resnet_layer(inputs=y, num_filters=64, kernel_size=1, conv_first=False)

# linear projection residual shortcut connection to match
# changed dims
x = resnet_layer(inputs=x, num_filters=64, kernel_size=1, strides=1, activation=None, batch_normalization=False)
x = keras.layers.add([x, y])

# bottleneck residual unit
y = resnet_layer(inputs=x, num_filters=16, kernel_size=1, strides=1, activation='relu', batch_normalization=True, conv_first=False)
y = resnet_layer(inputs=y, num_filters=16, conv_first=False)
y = resnet_layer(inputs=y, num_filters=64, kernel_size=1, conv_first=False)

x = keras.layers.add([x, y])

# stage 1

# bottleneck residual unit
y = resnet_layer(inputs=x, num_filters=64, kernel_size=1, strides=2, activation='relu', batch_normalization=True, conv_first=False)
y = resnet_layer(inputs=y, num_filters=64, conv_first=False)
y = resnet_layer(inputs=y, num_filters=128, kernel_size=1, conv_first=False)

# linear projection residual shortcut connection to match
# changed dims
x = resnet_layer(inputs=x, num_filters=128, kernel_size=1, strides=2, activation=None, batch_normalization=False)
x = keras.layers.add([x, y])

# bottleneck residual unit
y = resnet_layer(inputs=x, num_filters=64, kernel_size=1, strides=1, activation='relu', batch_normalization=True, conv_first=False)
y = resnet_layer(inputs=y, num_filters=64, conv_first=False)
y = resnet_layer(inputs=y, num_filters=128, kernel_size=1, conv_first=False)

x = keras.layers.add([x, y])

# stage 2

# bottleneck residual unit
y = resnet_layer(inputs=x, num_filters=128, kernel_size=1, strides=2, activation='relu', batch_normalization=True, conv_first=False)
y = resnet_layer(inputs=y, num_filters=128, conv_first=False)
y = resnet_layer(inputs=y, num_filters=256, kernel_size=1, conv_first=False)

# linear projection residual shortcut connection to match
# changed dims
x = resnet_layer(inputs=x,num_filters=256, kernel_size=1, strides=2, activation=None, batch_normalization=False)
x = keras.layers.add([x, y])

# bottleneck residual unit
y = resnet_layer(inputs=x, num_filters=128, kernel_size=1, strides=1, activation='relu', batch_normalization=True, conv_first=False)
y = resnet_layer(inputs=y, num_filters=128, conv_first=False)
y = resnet_layer(inputs=y, num_filters=256, kernel_size=1, conv_first=False)

x = keras.layers.add([x, y])


# Add classifier on top.
# v2 has BN-ReLU before Pooling
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = AveragePooling2D(pool_size=8, data_format='channels_first')(x)
y = Flatten()(x)
outputs = Dense(1, activation='sigmoid', kernel_initializer='he_normal')(y)

model = Model(inputs=inputs, outputs=outputs)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 1, 100, 100)  0                                            
__________________________________________________________________________________________________
conv2d_268 (Conv2D)             (None, 16, 100, 100) 160         input_17[0][0]                   
__________________________________________________________________________________________________
batch_normalization_239 (BatchN (None, 16, 100, 100) 400         conv2d_268[0][0]                 
__________________________________________________________________________________________________
activation_239 (Activation)     (None, 16, 100, 100) 0           batch_normalization_239[0][0]    
__________________________________________________________________________________________________
conv2d_269

In [28]:
input_shape = (1, 100, 100)
depth = 20
num_filters_in = 16
num_res_blocks = int((depth - 2) / 9)

inputs = Input(shape=input_shape)
# v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
x = resnet_layer(inputs=inputs,
                 num_filters=num_filters_in,
                 conv_first=True)

# Instantiate the stack of residual units
for stage in range(3):
    for res_block in range(num_res_blocks):
        activation = 'relu'
        batch_normalization = True
        strides = 1
        if stage == 0:
            num_filters_out = num_filters_in * 4
            if res_block == 0:  # first layer and first stage
                activation = None
                batch_normalization = False
        else:
            num_filters_out = num_filters_in * 2
            if res_block == 0:  # first layer but not first stage
                strides = 2    # downsample

        # bottleneck residual unit
        y = resnet_layer(inputs=x,
                         num_filters=num_filters_in,
                         kernel_size=1,
                         strides=strides,
                         activation=activation,
                         batch_normalization=batch_normalization,
                         conv_first=False)
        y = resnet_layer(inputs=y,
                         num_filters=num_filters_in,
                         conv_first=False)
        y = resnet_layer(inputs=y,
                         num_filters=num_filters_out,
                         kernel_size=1,
                         conv_first=False)
        if res_block == 0:
            # linear projection residual shortcut connection to match
            # changed dims
            x = resnet_layer(inputs=x,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             strides=strides,
                             activation=None,
                             batch_normalization=False)
        x = keras.layers.add([x, y])

    num_filters_in = num_filters_out

# Add classifier on top.
# v2 has BN-ReLU before Pooling
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = AveragePooling2D(pool_size=8, data_format='channels_first')(x)
y = Flatten()(x)
outputs = Dense(1,
                activation='softmax',
                kernel_initializer='he_normal')(y)

# Instantiate model.
model = Model(inputs=inputs, outputs=outputs)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 1, 100, 100)  0                                            
__________________________________________________________________________________________________
conv2d_312 (Conv2D)             (None, 16, 100, 100) 160         input_19[0][0]                   
__________________________________________________________________________________________________
batch_normalization_277 (BatchN (None, 16, 100, 100) 400         conv2d_312[0][0]                 
__________________________________________________________________________________________________
activation_277 (Activation)     (None, 16, 100, 100) 0           batch_normalization_277[0][0]    
__________________________________________________________________________________________________
conv2d_313

In [9]:
from keras import backend as K
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def squeeze_excite_block(input, ratio=16):
    ''' Create a channel-wise squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
    Returns: a keras tensor
    References
    -   [Squeeze and Excitation Networks](https://arxiv.org/abs/1709.01507)
    '''
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init._keras_shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x


In [10]:
#ResNetF with SE

input_shape = (100, 100, 2)

inputs = Input(shape=input_shape)  # output (1, 100, 100)
y = resnet_layer(inputs=inputs, num_filters=16, strides=1)  # output (16, 100, 100)

# stack 0
x = resnet_layer(inputs=y, num_filters=16, strides=1)
x = resnet_layer(inputs=x, num_filters=16, strides=1, activation=None)
# squeeze and excite block
x = squeeze_excite_block(x)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=16, strides=1)
x = resnet_layer(inputs=x, num_filters=16, strides=1, activation=None)
# squeeze and excite block
x = squeeze_excite_block(x)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=16, strides=1)
x = resnet_layer(inputs=x, num_filters=16, strides=1, activation=None)
# squeeze and excite block
x = squeeze_excite_block(x)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

# stack 1
x = resnet_layer(inputs=y, num_filters=32, strides=2)
x = resnet_layer(inputs=x, num_filters=32, strides=1, activation=None)
# linear projection
y = resnet_layer(inputs=y, num_filters=32, kernel_size=1, strides=2, activation=None, batch_normalization=False)
# squeeze and excite block
x = squeeze_excite_block(x)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=32, strides=1)
x = resnet_layer(inputs=x, num_filters=32, strides=1, activation=None)
# squeeze and excite block
x = squeeze_excite_block(x)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=32, strides=1)
x = resnet_layer(inputs=x, num_filters=32, strides=1, activation=None)
# squeeze and excite block
x = squeeze_excite_block(x)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

# stack 2
x = resnet_layer(inputs=y, num_filters=64, strides=2)
x = resnet_layer(inputs=x, num_filters=64, strides=1, activation=None)
# linear projection
y = resnet_layer(inputs=y, num_filters=64, kernel_size=1, strides=2, activation=None, batch_normalization=False)
# squeeze and excite block
x = squeeze_excite_block(x)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=64, strides=1)
x = resnet_layer(inputs=x, num_filters=64, strides=1, activation=None)
# squeeze and excite block
x = squeeze_excite_block(x)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=64, strides=1)
x = resnet_layer(inputs=x, num_filters=64, strides=1, activation=None)
# squeeze and excite block
x = squeeze_excite_block(x)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

# stack 2
x = resnet_layer(inputs=y, num_filters=128, strides=2)
x = resnet_layer(inputs=x, num_filters=128, strides=1, activation=None)
# linear projection
y = resnet_layer(inputs=y, num_filters=128, kernel_size=1, strides=2, activation=None,
                 batch_normalization=False)
# squeeze and excite block
x = squeeze_excite_block(x)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=128, strides=1)
x = resnet_layer(inputs=x, num_filters=128, strides=1, activation=None)
# squeeze and excite block
x = squeeze_excite_block(x)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=128, strides=1)
x = resnet_layer(inputs=x, num_filters=128, strides=1, activation=None)
# squeeze and excite block
x = squeeze_excite_block(x)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = AveragePooling2D(pool_size=2)(y)
y = Flatten()(x)
outputs = Dense(1, activation='sigmoid', kernel_initializer='he_normal')(y)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 100, 100, 2)  0                                            
__________________________________________________________________________________________________
conv2d_63 (Conv2D)              (None, 100, 100, 16) 304         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_57 (BatchNo (None, 100, 100, 16) 64          conv2d_63[0][0]                  
__________________________________________________________________________________________________
activation_55 (Activation)      (None, 100, 100, 16) 0           batch_normalization_57[0][0]     
__________________________________________________________________________________________________
conv2d_64 

In [ ]:
#ResnetH

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


input_shape = (2, self.img_rows, self.img_cols)

inputs = Input(shape=input_shape)  # output (1, 100, 100)
y = resnet_layer(inputs=inputs, num_filters=8, strides=1)  # output (16, 100, 100)

# stack 0
x = resnet_layer(inputs=y, num_filters=8, strides=1)
x = resnet_layer(inputs=x, num_filters=8, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=8, strides=1)
x = resnet_layer(inputs=x, num_filters=8, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=8, strides=1)
x = resnet_layer(inputs=x, num_filters=8, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

# stack 0
x = resnet_layer(inputs=y, num_filters=16, strides=2)
x = resnet_layer(inputs=x, num_filters=16, strides=1, activation=None)
# linear projection
y = resnet_layer(inputs=y, num_filters=16, kernel_size=1, strides=2, activation=None, batch_normalization=False)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=16, strides=1)
x = resnet_layer(inputs=x, num_filters=16, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=16, strides=1)
x = resnet_layer(inputs=x, num_filters=16, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=16, strides=1)
x = resnet_layer(inputs=x, num_filters=16, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

# stack 1
x = resnet_layer(inputs=y, num_filters=32, strides=2)
x = resnet_layer(inputs=x, num_filters=32, strides=1, activation=None)
# linear projection
y = resnet_layer(inputs=y, num_filters=32, kernel_size=1, strides=2, activation=None, batch_normalization=False)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=32, strides=1)
x = resnet_layer(inputs=x, num_filters=32, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=32, strides=1)
x = resnet_layer(inputs=x, num_filters=32, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=32, strides=1)
x = resnet_layer(inputs=x, num_filters=32, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=32, strides=1)
x = resnet_layer(inputs=x, num_filters=32, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=32, strides=1)
x = resnet_layer(inputs=x, num_filters=32, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

# stack 2
x = resnet_layer(inputs=y, num_filters=64, strides=2)
x = resnet_layer(inputs=x, num_filters=64, strides=1, activation=None)
# linear projection
y = resnet_layer(inputs=y, num_filters=64, kernel_size=1, strides=2, activation=None, batch_normalization=False)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=64, strides=1)
x = resnet_layer(inputs=x, num_filters=64, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=64, strides=1)
x = resnet_layer(inputs=x, num_filters=64, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=64, strides=1)
x = resnet_layer(inputs=x, num_filters=64, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=64, strides=1)
x = resnet_layer(inputs=x, num_filters=64, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=64, strides=1)
x = resnet_layer(inputs=x, num_filters=64, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

# stack 2
x = resnet_layer(inputs=y, num_filters=128, strides=2)
x = resnet_layer(inputs=x, num_filters=128, strides=1, activation=None)
# linear projection
y = resnet_layer(inputs=y, num_filters=128, kernel_size=1, strides=2, activation=None,
                 batch_normalization=False)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=128, strides=1)
x = resnet_layer(inputs=x, num_filters=128, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=128, strides=1)
x = resnet_layer(inputs=x, num_filters=128, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=128, strides=1)
x = resnet_layer(inputs=x, num_filters=128, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=128, strides=1)
x = resnet_layer(inputs=x, num_filters=128, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=128, strides=1)
x = resnet_layer(inputs=x, num_filters=128, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=128, strides=1)
x = resnet_layer(inputs=x, num_filters=128, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=128, strides=1)
x = resnet_layer(inputs=x, num_filters=128, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = resnet_layer(inputs=y, num_filters=128, strides=1)
x = resnet_layer(inputs=x, num_filters=128, strides=1, activation=None)
x = keras.layers.add([x, y])
y = Activation('relu')(x)

x = AveragePooling2D(pool_size=2)(y)
y = Flatten()(x)
outputs = Dense(1, activation='linear', kernel_initializer='he_normal')(y)
model = Model(inputs=inputs, outputs=outputs)